# Cryptographic Hashes

Finally! A notebook all about an infosec! We are going to explore cryptographic hashes.

## Hash function

A hash function is a function that maps an input string of arbitrary length to a binary string of fixed length. So a hash function takes just one input argument. We can think of the output as just a binary number. When we input a string, the function returns a number. If we input in the same string again, we get the identical number out. Hashing different strings may or may not result in the same number, but the length of the output will always be the same.

Let's look at an example hash function I call "badHash". The input is a string, and the output is an 8-bit number. We'll represent the output in hex, so it'll be two hex digits long. The function takes the XOR of all of the characters in the input string to produce its output.

In [ ]:
def badHash(inputString):
    digest=0
    for character in inputString:
        digest=digest^ord(character)
    return digest

We can compute the badHash message digest of a couple strings.

In [ ]:
print(hex(badHash("HELLO")))
print(hex(badHash("cat")))

Note that badHash uses the "accumulator pattern" we saw earlier. We start with the variable "digest" storing zero. As we loop through each character in the variable "inputString", we XOR its Unicode value with the previous value of "digest". Then we use that result to update the value of "digest".

## Possible outputs

badHash produces 8-bits of output. How many distinct output values are possible? In other words, how many distinct binary strings are there of length 8? Let's try to figure this out by starting small. For a binary string of length 1, we have 2 possibilities "0" and "1". For a binary string of length 2, we have 4 possibilities "00", "01", "10", and "11". For a binary string of length 3, we have 8 possibilities "000", "001", "010", "011", "100", "101", "110", "111". This is taking too long. Let's try to generalize!

Whenever we add a new bit, the new bit can either be "0" or "1". If the new bit is "0", the remaining bits have the old number of variations. If the new bit is "1", the remaining bits also have the old number of variations. Each time we add a new bit, we double the number of variations.

In other words, for a binary string of length $n$, there are $2^n$ possible values. For 8-bits, that's $2^8=256$. So badHash only has 256 different possible hash digests!

In [ ]:
2**8

## Collisions

When two strings have the same hash output, we call this a **collision**. Below, we have an example of a hash collision. These two strings produce the same hash output.

In [ ]:
print(hex(badHash("HELLO")))
print(hex(badHash("LISTEN SILENT HELLO")))

From the very definition of a hash function, collisions can always happen. We said we can input any string. Well, there are infinitely many different strings. The output of a hash function, however, is always a fixed-length. A fix-length string of n bits only has $2^n$ different possibilities. No matter how big $n$ is, that number is still finite, and an infinite number is *definitely* bigger than a finite number.

Infinite inputs. Finite outputs. Clearly some inputs will share the same output. Hash collisions are inevitable.

## Cryptographic Hash Functions

Cryptographic hash functions are hash functions engineered to have three special properties, and badHash has none of them. It really lives up to its name. We're going to learn about the three properties by attacking badHash. Welcome to the adversarial mindset!

### Property 1: Preimage resistance

An attacker should not be able to find an input that produces a given hash output. In other words, we want our hash function to be a one-way function. Looking at the output shouldn't tell an attacker anything about the input. Without the input string, the output should just look like a random number.

Let's attack badHash. We want to find a string that will produce a desired output when we feed it into badHash. This turns out to be very simple. All we need to do is find the unicode character whose binary encoding matches the desired output. We feed that character in by itself. Since we don't have any other characters in our input string, it doesn't get XORed with anything. The badHash output is just the Unicode for our input character.

Here's a function to perform a preimage attack against badHash. It automatically finds a string with the desired hash output.

In [ ]:
def preimage(n):
    return chr(n)

desired=125
x=preimage(desired)
print(x)
print(badHash(x))

Even if we didn't have access to the Unicode table or the chr built-in function, there are only $2^8=256$ possible outputs. That's a very small number of possible outputs. We could just guess input strings until we find one that has the desired hash output. This is called a **brute force attack**. Here's a brute force attack to find a string that not only has the desired hash output, but also starts with a specific message. 

Note: This uses some coding tricks we won't learn until later. Don't worry if you don't understand exactly how this works.

In [ ]:
def preimage2(n):
    guess=0
    message="Honestly at this point we're just showing off."
    while(badHash(message+chr(guess))!=n):
        guess+=1
    return message+chr(guess)
    
desired=200
x=preimage2(desired)
print(x)
print(badHash(x))

So badHash does not have preimage resistance.

### Property 2: Second preimage resistance

Given a string s, an attacker should not be able to find a different string t that has the same digest as s. Without this property, an attacker can forge a message with the same digest as our message. We want to be able to trust that a string having the same digest as s means that string **is** s.

Once again, we'll see that badHash just isn't up to the challenge. Previously, we learned that a number XORed with itself is always 0. Try it yourself if you need a refresher.

In [ ]:
x=5
print(x^x)

Since badHash is based on XOR, we can take advantage of this property. What happens when we repeat characters in badHash?

In [ ]:
print(badHash("nn"))

We get zero, because x XOR x =0. In fact, this will work no matter how many characters are in the string or what order they're in. As long as they appear twice, they cancel each other out and produce 0.

In [ ]:
print(badHash("CABBACDD"))

This is a vulnerability in badHash. If we add repeating characters, they cancel out and produce zero. In other words, they have no impact on the output. We can exploit this vulnerability. Given a string s, we can find another string t with a matching hash digest. For example, if s was "HELLO"...

In [ ]:
s="HELLO"
print(badHash(s))
t="HELLOXX"
print(badHash(t))

We can easily produce a t with a matching hash digest. In fact, we can write a function to automatically perform this attack for any string s.

In [ ]:
def secondPreimage(s):
    return s+"XX"

s="My top secret message."
t=secondPreimage(s)
print(badHash(s),badHash(t))

If we're clever, we can use this property to engineer a message with a very different meaning than the original.

In [ ]:
s="DANGER! WAIT! DO NOT ATTACK!"
t="NO DANGER! DO NOT WAIT ON ME! ATTACK EM! ATTACK! ATTACK!"
badHash(s),badHash(t)

### Property 3: Collision resistance

This property is more stringent than the previous two. We don't even want an attacker to be able to find two strings s and t that have the same hash digest. In this case, we're letting the attacker pick **both** strings. This gives Mallory a lot more power. It takes a very well designed cryptographic hash function to be collision resistant.

We saw that badHash doesn't have the previous two properties. Do you think it has collision resistance? Obviously not! We've already engineered many collisions for badHash. It would be trivial to engineer more. In fact, here are 10 ridiculously long words I found in the dictionary that all share the same hash digest as 'infosec': 'autobasidiomycetous', 'choledocholithotomy', 'contemporaneousness', 'cystopyelonephritis', 'hypsidolichocephaly', 'incomprehensibility', 'otorhinolaryngology', 'representationalism', 'uncircumscribedness', and 'unconsequentialness'.

In [ ]:
print(badHash('incomprehensibility'),badHash('representationalism'),badHash('autobasidiomycetous'))

Since badHash is so easy to attack, I'm going to make a slightly improved hash that still produces 8-bit digests.

In [ ]:
def lessBadHash(inputString):
    digest=0x3C
    sponge=0xA5
    for character in inputString:
        x=ord(character)*19+sponge*23
        digest=(digest^x)%509
        sponge=x
    return digest%256

This new hash function is very complicated. Any mathematical weaknesses in this hash are much less obvious. Can we still find collisions? Yes!

## Birthday Attack

If the attacker can choose **both** hash input strings, they can take advantage of the Birthday Paradox. As the size of a set increases linearly, the number of pairs increases as the square. For a set of size n, the number of pairs is $\binom{n}{2}=\frac{n(n-1)}{2}$. If 5 friends are hanging out, there are $\frac{5*4}{2}=10$ different possible conversation partners. With 6 friends, there are $\frac{6*5}{2}=15$.

How does this help the attacker? There are $2^8=256$ possible lessBadHash digests. This means the chance that two strings share a common lessBadHash digest is $\frac{1}{256}$. How many strings do we need to guess before we find two with a matching digest? You might guess 256 or maybe 128, but it's actually requires much less effort.

The Birthday Paradox tells us that our intuition is wrong. It will actually only take $\sqrt2^{8}=2^{8/2}=2^4=16$. We only need to try about 16 strings! In practice, we need to try a bit more than that, but nowhere near 256. Do you see the collision?

Note: If you're curious, the **str** built-in function I use below just turns an integer into a string (e.g. 10 becomes "10".)

In [ ]:
for i in range(30):
    print(lessBadHash(str(i)))

Did you find the hash collision in all that output? It's there. The birthday paradox is a tricky problem. To defend against it, we need to dramatically increase the size of our hash digests.

## Exercises

Below, you'll find the exercises for this notebook. If you struggle, remember to take breaks and ask for help when you need them.

1) Find a string whose badHash digest is 0x55.

2) Find a string whose badHash digest matches that of the string "tiger".

3) Create two English messages s and t with the same badHash digests, but whose meaning is very different. Be creative!

4) Find a pair of strings with matching lessBadHash digests. Your pair should be **different** than the one I found above.